In [61]:
import os
from scipy.stats import zscore # Z-normalization
from TSB_AD.model_wrapper import run_Unsupervise_AD, run_Semisupervise_AD
from TSB_AD.evaluation.metrics import get_metrics
import torch
import pandas as pd
import numpy as np

In [62]:
print("CUDA available: ", torch.cuda.is_available())
print("cuDNN version: ", torch.backends.cudnn.version())
if torch.cuda.is_available():
    print("GPU device count: ", torch.cuda.device_count())
    print("GPU device name: ", torch.cuda.get_device_name(0))

CUDA available:  True
cuDNN version:  90100
GPU device count:  1
GPU device name:  NVIDIA GeForce RTX 2060


# For lightcurve datasets

In [63]:
# Paths
dataset_folder = os.path.join('..', 'datasets', 'light_curves')
mdwarfs_filename   = '14_mdwarfs.csv'
kepler_filename    = 'kepler_H3_D120_3456_3466.txt'

# Example TSAD detectors
semisupervised_detector   = 'AnomalyTransformer'
unsupervised_detector    = 'Series2Graph'
result = []

In [64]:
# Kepler

# Extract flare metadata from filename
#   Format: kepler_H<height>_D<duration>_<rising_start>_<decay_start>.txt
meta_parts         = kepler_filename.rstrip('.txt').split('_')
duration_days      = int(meta_parts[2].lstrip('D'))
rising_start_idx   = int(meta_parts[3])
decay_start_idx    = int(meta_parts[4])


file_path = os.path.join(dataset_folder, kepler_filename)
with open(file_path, 'r') as f:
    raw_flux = [float(line.strip()) for line in f]
kepler_light_curve = (
    zscore(np.array(raw_flux))
    .astype(np.float64)
    .reshape(-1, 1)
)

# Build true-label masks
true_rising_labels = np.zeros(len(kepler_light_curve), dtype=int)
true_rising_labels[rising_start_idx:decay_start_idx] = 1

true_full_labels   = np.zeros(len(kepler_light_curve), dtype=int)
true_full_labels[rising_start_idx:rising_start_idx + duration_days] = 1

# Run the unsupervised anomaly detector
anomaly_scores = run_Unsupervise_AD(unsupervised_detector, kepler_light_curve)

# Evaluate and collect results for each phase
for scores, true_labels, phase in [
    (anomaly_scores, true_rising_labels, 'rising'),
    (anomaly_scores, true_full_labels,   'full'),
]:
    metrics = get_metrics(scores, true_labels)
    metrics.update({
        'detector_name':  unsupervised_detector,
        'detector_type':  'unsupervised',
        'file_name':      kepler_filename,
        'anomaly_phase':  phase,
        'flare_type': 'Kepler',
    })
    result.append(metrics)

D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\TSB_AD\models\Series2Graph.py:430: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [65]:
idx_training = 500
train_data = kepler_light_curve[:idx_training]
test_data = kepler_light_curve[idx_training:]

# Run the unsupervised anomaly detector
anomaly_scores = run_Semisupervise_AD(semisupervised_detector, train_data, test_data)

# Evaluate and collect results for each phase
for scores, true_labels, phase in [
    (anomaly_scores, true_rising_labels[idx_training:], 'rising'),
    (anomaly_scores, true_full_labels[idx_training:],   'full'),
]:
    metrics = get_metrics(scores, true_labels)
    metrics.update({
        'detector_name':  semisupervised_detector,
        'detector_type':  'semisupervised',
        'file_name':      kepler_filename,
        'anomaly_phase':  phase,
        'flare_type': 'Kepler',
    })
    result.append(metrics)

----- Using GPU NVIDIA GeForce RTX 2060 -----
======================TRAIN MODE======================


Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00, 15.05it/s, loss1=-24.2, loss2=24.6]


Updating learning rate to 0.0001


Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00, 15.12it/s, loss1=-24.8, loss2=25.2]


EarlyStopping counter: 1 out of 7
Updating learning rate to 5e-05


...

Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00, 15.89it/s, loss1=-26.3, loss2=26.4]


EarlyStopping counter: 6 out of 7
Updating learning rate to 1.5625e-06


Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00, 16.08it/s, loss1=-26.3, loss2=26.4]
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


EarlyStopping counter: 7 out of 7
Early stopping
======================TEST MODE======================


100%|██████████| 29/29 [00:01<00:00, 14.80it/s]
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Mdwarfs : Unsupervised

In [66]:
# Mdwarfs
rising_start_time = -20       # start of rising phase
flare_end_time    = 30        # end of full flare duration

df = pd.read_csv(os.path.join(dataset_folder, mdwarfs_filename))

# Build true-label masks
df['label_rising'] = (
    (df['timestamps'] >= rising_start_time) & (df['timestamps'] <= 0)
).astype(int)
df['label_full'] = (
    (df['timestamps'] >= rising_start_time) & (df['timestamps'] <= flare_end_time)
).astype(int)


mdwarfs_light_curve = df['nor'].astype(float).values.reshape(-1, 1)
true_rising_labels  = df['label_rising'].to_numpy()
true_full_labels    = df['label_full'].to_numpy()

# Run the unsupervised anomaly detector
anomaly_scores = run_Unsupervise_AD(unsupervised_detector, mdwarfs_light_curve)

# Evaluate and collect results for each phase
for scores, true_labels, phase in [
    (anomaly_scores, true_rising_labels, 'rising'),
    (anomaly_scores, true_full_labels,   'full'),
]:
    metrics = get_metrics(scores, true_labels)
    metrics.update({
        'detector_name':  unsupervised_detector,
        'detector_type':  'unsupervised',
        'file_name':      mdwarfs_filename,
        'anomaly_phase':  phase,
        'flare_type': 'Mdwarfs',
    })
    result.append(metrics)

D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\TSB_AD\models\Series2Graph.py:430: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df = df.fillna(method='bfill')
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# Mdwarfs : semi-supervised

In [67]:
timestamps_array = df['timestamps'].to_numpy()
idx_training = round(len(timestamps_array)*0.3)

train_data = mdwarfs_light_curve[:idx_training]
test_data = mdwarfs_light_curve[idx_training:]

# Run the unsupervised anomaly detector
anomaly_scores = run_Semisupervise_AD(semisupervised_detector, train_data, test_data)

# Evaluate and collect results for each phase
for scores, true_labels, phase in [
    (anomaly_scores, true_rising_labels[idx_training:], 'rising'),
    (anomaly_scores, true_full_labels[idx_training:],   'full'),
]:
    metrics = get_metrics(scores, true_labels)
    metrics.update({
        'detector_name':  semisupervised_detector,
        'detector_type':  'semisupervised',
        'file_name':      mdwarfs_filename,
        'anomaly_phase':  phase,
        'flare_type': 'Mdwarfs',
    })
    result.append(metrics)


----- Using GPU NVIDIA GeForce RTX 2060 -----
======================TRAIN MODE======================


Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00,  9.34it/s, loss1=-26.3, loss2=27]


Updating learning rate to 0.0001


Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00,  9.03it/s, loss1=-30, loss2=30.6]


EarlyStopping counter: 1 out of 7
Updating learning rate to 5e-05


...

Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00,  9.11it/s, loss1=-35.3, loss2=35.5]


EarlyStopping counter: 6 out of 7
Updating learning rate to 1.5625e-06


Validation Epoch: : 100%|██████████| 1/1 [00:00<00:00,  8.84it/s, loss1=-35.4, loss2=35.6]
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\torch\nn\_reduction.py:51: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


EarlyStopping counter: 7 out of 7
Early stopping
======================TEST MODE======================


100%|██████████| 17/17 [00:01<00:00, 14.72it/s]
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
D:\git_project\ts-mining-light-curve\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [68]:
df_LC = pd.DataFrame(result).reset_index()
df_LC

,index,AUC-PR,AUC-ROC,VUS-PR,VUS-ROC,Standard-F1,PA-F1,Event-based-F1,R-based-F1,Affiliation-F,detector_name,detector_type,file_name,anomaly_phase,flare_type
0,0,0.008017,0.822217,0.048265,0.916547,0.021621,0.070922,0.021362,0.034146,0.906999,Series2Graph,unsupervised,kepler_H3_D120_3456_3466.txt,rising,Kepler
1,1,0.089237,0.707723,0.092150,0.745007,0.170342,0.995851,0.666667,0.292453,0.918881,Series2Graph,unsupervised,kepler_H3_D120_3456_3466.txt,full,Kepler
2,2,0.002660,0.497599,0.039756,0.544807,0.005306,0.000000,0.000000,0.000000,0.981245,AnomalyTransformer,semisupervised,kepler_H3_D120_3456_3466.txt,rising,Kepler
3,3,0.041114,0.506149,0.051874,0.509317,0.061871,1.000000,1.000000,0.342541,0.991232,AnomalyTransformer,semisupervised,kepler_H3_D120_3456_3466.txt,full,Kepler
4,4,0.148374,0.979612,0.391537,0.990970,0.324321,0.421053,0.317757,0.333365,0.988382,Series2Graph,unsupervised,14_mdwarfs.csv,rising,Mdwarfs
5,5,0.189588,0.961495,0.418762,0.981928,0.368790,0.645161,0.434783,0.346821,0.984853,Series2Graph,unsupervised,14_mdwarfs.csv,full,Mdwarfs
6,6,0.008818,0.496664,0.027780,0.506320,0.017482,0.000000,0.000000,0.000000,0.833368,AnomalyTransformer,semisupervised,14_mdwarfs.csv,rising,Mdwarfs
7,7,0.022046,0.496619,0.041396,0.509751,0.043140,0.000000,0.000000,0.000000,0.844709,AnomalyTransformer,semisupervised,14_mdwarfs.csv,full,Mdwarfs
